1. **`Matcher(nlp.vocab)`**

   * **Qué hace:** crea un objeto *Matcher*, que permite definir **patrones de tokens** para encontrar secuencias específicas en el texto.
   * **Parámetro `nlp.vocab`:** le da acceso al vocabulario del modelo (lexemas, atributos de tokens, etc.).

2. **`matcher.add("NombreDelPatrón", [pattern])`**

   * **Qué hace:** agrega un patrón de coincidencia al *Matcher*.
   * **Cómo funciona el patrón:** se define como una lista de diccionarios con condiciones sobre tokens (`IS_CURRENCY`, `LIKE_NUM`, `LOWER`, `LEMMA`, etc.).
   * **Ejemplo de uso:**

     * `pattern1` busca símbolos de moneda seguidos de números.
     * `pattern2` busca números seguidos de palabras como *pesos*, *dólares*, etc.

3. **`matches = matcher(doc)`**

   * **Qué hace:** aplica los patrones definidos al `Doc`.
   * **Devuelve:** una lista de tuplas `(match_id, start, end)` → el nombre del patrón y la posición de los tokens coincidentes.
   * **Cómo se usa:** con `doc[start:end]` se obtiene el `Span` de texto que coincide.

In [2]:
import spacy
from spacy.matcher import Matcher

# Cargar el modelo de español
nlp = spacy.load("es_core_news_lg")

# Inicializar el Matcher
matcher = Matcher(nlp.vocab)

# ----------------- DEFINICIÓN DE PATRONES -----------------

# Patrón 1: Para montos que comienzan con un símbolo de moneda.
# Ejemplo: "$250.000", "€1.5 millones"
pattern1 = [
    {"IS_CURRENCY": True},  # Un símbolo de moneda
    {"LIKE_NUM": True, "OP": "+"},  # Uno o más tokens numéricos
    {"LOWER": {"IN": ["mil", "millón", "millones"]}, "OP": "?"}  # Un magnitud opcional
]

# Patrón 2: Para montos que terminan con el nombre de la moneda.
# Ejemplo: "50 pesos", "cien mil dólares", "1.5 millones de euros"
pattern2 = [
    {"LIKE_NUM": True, "OP": "+"},  # Uno o más tokens numéricos
    {"LOWER": {"IN": ["mil", "millón", "millones"]}, "OP": "?"},  # Una magnitud opcional
    {"LEMMA": "de", "OP": "?"},  # La palabra opcional "de"
    {"LOWER": {"IN": ["dólar", "dólares", "peso", "pesos", "euro", "euros"]}}  # El nombre de la moneda
]

# Añadir los patrones al matcher con nombres únicos
matcher.add("MoneySymbolFirst", [pattern1])
matcher.add("MoneyNameLast", [pattern2])

# ----------------- PROCESAMIENTO Y BÚSQUEDA -----------------

# Texto con diferentes formatos de montos de dinero
texto = """
    El informe financiero de Innovaciones Globales S.A. de C.V. para el cierre del año fiscal detalla un panorama de crecimiento sostenido y expansión estratégica en múltiples mercados. La junta directiva aprobó una inyección de capital significativa, donde la empresa invirtió $250.000 en el primer trimestre, destinados principalmente a la modernización de su infraestructura tecnológica y al fortalecimiento del equipo de investigación y desarrollo. Esta inversión inicial ya muestra un retorno preliminar positivo, optimizando la eficiencia operativa en un 15% y sentando las bases para futuros lanzamientos de productos.

El enfoque principal de la expansión internacional se centró en el mercado europeo. El presupuesto para el siguiente proyecto es de 1.5 millones de euros, asignado a la apertura de una nueva sede en Berlín, Alemania. De este monto, el 40% se ha destinado a la adquisición y acondicionamiento de las oficinas, un 30% a la contratación de personal clave, incluyendo ingenieros y especialistas en ventas, y el 30% restante se mantiene como capital de trabajo para cubrir los gastos operativos iniciales durante los primeros dos años. Se proyecta que esta nueva filial comience a generar ingresos netos a partir del tercer trimestre del próximo año, consolidando nuestra presencia en la Unión Europea.

Paralelamente, las operaciones en América Latina continuaron su consolidación. Aunque se registraron numerosos gastos menores, como cuando se pagaron 50 pesos mexicanos por el servicio de mensajería local, el grueso de los gastos operativos en México ascendió a 1.2 millones de pesos mensuales, cubriendo nóminas, logística y pagos a proveedores. Se logró un acuerdo estratégico con un distribuidor local en Brasil, lo que representó una transacción inicial de 800,000 reales brasileños y abre un nuevo canal de ventas con un potencial de mercado considerable.

Para asegurar el cumplimiento normativo en nuestras nuevas jurisdicciones, se contrataron los servicios de una prestigiosa firma de consultoría con sede en Londres. El costo de esta asesoría legal y financiera fue de £75,000 libras esterlinas, una inversión crucial para mitigar riesgos y garantizar una entrada fluida en el mercado británico. Durante una visita de prospección a Nueva York, se realizaron gastos menores de viáticos, y en una anécdota del equipo, sobraron cien dólares del fondo para imprevistos, que fueron debidamente reintegrados a la tesorería de la compañía.

Mirando hacia el futuro, la compañía anticipa un crecimiento de ingresos del 25% para el próximo año fiscal, impulsado por la consolidación de las nuevas operaciones y el lanzamiento de la línea de productos "Quantum". La dirección financiera está evaluando la posibilidad de una nueva ronda de financiamiento para explorar la entrada al mercado asiático, con un estudio de viabilidad centrado en Singapur y Corea del Sur. Se estima una necesidad de capital de aproximadamente 3 millones de dólares para esta siguiente fase de expansión global, reafirmando el compromiso de Innovaciones Globales con su visión de liderazgo en el sector tecnológico a nivel mundial. El flujo de caja se mantiene saludable, permitiendo a la empresa ejecutar su ambiciosa hoja de ruta con confianza y solidez financiera.
"""

# Procesar el texto
doc = nlp(texto)

# Encontrar todas las coincidencias
matches = matcher(doc)

# Imprimir los resultados de forma detallada
print(f"Texto analizado: \"{texto}\"")
print("-" * 30)
print(f"Se encontraron {len(matches)} coincidencias de dinero:")

for match_id, start, end in matches:
    # Obtener el nombre del patrón que encontró la coincidencia
    pattern_name = nlp.vocab.strings[match_id]
    # Obtener el trozo de texto (Span)
    span = doc[start:end]
    print(f"- Patrón '{pattern_name}': {span.text}")

Texto analizado: "
    El informe financiero de Innovaciones Globales S.A. de C.V. para el cierre del año fiscal detalla un panorama de crecimiento sostenido y expansión estratégica en múltiples mercados. La junta directiva aprobó una inyección de capital significativa, donde la empresa invirtió $250.000 en el primer trimestre, destinados principalmente a la modernización de su infraestructura tecnológica y al fortalecimiento del equipo de investigación y desarrollo. Esta inversión inicial ya muestra un retorno preliminar positivo, optimizando la eficiencia operativa en un 15% y sentando las bases para futuros lanzamientos de productos.

El enfoque principal de la expansión internacional se centró en el mercado europeo. El presupuesto para el siguiente proyecto es de 1.5 millones de euros, asignado a la apertura de una nueva sede en Berlín, Alemania. De este monto, el 40% se ha destinado a la adquisición y acondicionamiento de las oficinas, un 30% a la contratación de personal clave, i

4. **`displacy.serve(doc, style="dep")`**

   * **Qué hace:** abre un servidor local (`http://localhost:5000`) para visualizar gráficamente el análisis.
   * **Parámetro `style`:**

     * `"dep"` → muestra las dependencias sintácticas (quién depende de quién en la oración).
     * `"ent"` → muestra las entidades nombradas.
   * **Para qué sirve:** explorar de forma visual la salida del modelo NLP.

In [ ]:
import spacy
from spacy import displacy

nlp = spacy.load("es_core_news_md")

texto = "La gata negra perseguía un ratón pequeño."

doc = nlp(texto)

print("Iniciando visualización en http://localhost:5000")
displacy.serve(doc, style="dep")